In [2]:
import re
import sys
sys.path.append('../')
import os
import pandas as pd
from collections import Counter, defaultdict
import datetime
from tqdm import tqdm
import string
from copy import deepcopy

import hashlib
import time

pd.set_option('display.max_rows', None)

In [ ]:
import os

def split_files(log_file, structured_file, output_dir, sizes):
    os.makedirs(output_dir, exist_ok=True)
    
# Dùng encoding utf-8, nếu vẫn lỗi thì đổi sang ISO-8859-1
    try:
        with open(log_file, 'r', encoding='utf-8') as f:
            log_lines = f.readlines()
    except UnicodeDecodeError:
        with open(log_file, 'r', encoding='ISO-8859-1') as f:
            log_lines = f.readlines()

    try:
        with open(structured_file, 'r', encoding='utf-8') as f:
            structured_lines = f.readlines()
    except UnicodeDecodeError:
        with open(structured_file, 'r', encoding='ISO-8859-1') as f:
            structured_lines = f.readlines()
    header = structured_lines[0]
    structured_data = structured_lines[1:]
    # Kiểm tra số dòng có trùng không
    print(len(log_lines), "dòng trong log file.")
    print(len(structured_data), "dòng trong structured file.")
    assert len(log_lines) == len(structured_data), "Số dòng log và structured không khớp!"

    for size in sizes:
        count = size * 1000
        if count > len(log_lines):
            print(f"Bỏ qua {size}k vì vượt quá số dòng ({len(log_lines)}).")
            continue

        
        size_str = f"{size}k" if size < 1000 else f"{size // 1000}M"
        log_outfile = os.path.join(output_dir, f"Thunderbird_{size_str}.log")
        structured_outfile = os.path.join(output_dir, f"Thunderbird_{size_str}.log_structured.csv")

        # Ghi dữ liệu
        with open(log_outfile, 'w', encoding='utf-8') as f:
            f.writelines(log_lines[:count])
        with open(structured_outfile, 'w', encoding='utf-8') as f:
            f.write(header)
            f.writelines(structured_data[:count])

        print(f"Đã tạo {log_outfile} và {structured_outfile} với {count} dòng.")

# --- Cấu hình ---
log_file = "./logsfull/Thunderbird/Thunderbird_full.log"
structured_file = "./logsfull/Thunderbird/Thunderbird_full.log_structured.csv"
output_dir = "./logsplit/Thunderbird"
sizes = [2, 10, 100, 500, 1000, 2000, 4000]  # đơn vị: nghìn dòng (k)

# --- Chạy ---
split_files(log_file, structured_file, output_dir, sizes)

16601745 dòng trong log file.
16601745 dòng trong structured file.
Đã tạo ./logsplit/Thunderbird\Thunderbird_2k.log và ./logsplit/Thunderbird\Thunderbird_2k.log_structured.csv với 2000 dòng.
Đã tạo ./logsplit/Thunderbird\Thunderbird_10k.log và ./logsplit/Thunderbird\Thunderbird_10k.log_structured.csv với 10000 dòng.
Đã tạo ./logsplit/Thunderbird\Thunderbird_100k.log và ./logsplit/Thunderbird\Thunderbird_100k.log_structured.csv với 100000 dòng.
Đã tạo ./logsplit/Thunderbird\Thunderbird_500k.log và ./logsplit/Thunderbird\Thunderbird_500k.log_structured.csv với 500000 dòng.
Đã tạo ./logsplit/Thunderbird\Thunderbird_1M.log và ./logsplit/Thunderbird\Thunderbird_1M.log_structured.csv với 1000000 dòng.
Đã tạo ./logsplit/Thunderbird\Thunderbird_2M.log và ./logsplit/Thunderbird\Thunderbird_2M.log_structured.csv với 2000000 dòng.
Đã tạo ./logsplit/Thunderbird\Thunderbird_4M.log và ./logsplit/Thunderbird\Thunderbird_4M.log_structured.csv với 4000000 dòng.


In [8]:
pd.set_option('display.max_colwidth', None)

def printTemplate(csv_file, num_examples=10):
    df = pd.read_csv(csv_file, encoding='utf-8')
    df = df[:4000000]
    unique_templates = df[['EventTemplate', 'EventId']].drop_duplicates()
    print("Tổng số EventTemplate duy nhất:", len(unique_templates))
    print("=" * 50)
    for _, row in unique_templates.iterrows():
        template = row['EventTemplate']
        event_id = row['EventId']

        matched_contents = df[df['EventTemplate'] == template]['Content'].head(num_examples).tolist()

        print(f"EventID: {event_id}")
        print(f"EventTemplate: {template}")
        print("Contents:")
        for i, content in enumerate(matched_contents, 1):
            print(f"{i:2d}. {content}")
        print("-" * 50)

csv_file = "./logsfull/Thunderbird/Thunderbird_full.log_structured.csv"
printTemplate(csv_file, num_examples=10)

Tổng số EventTemplate duy nhất: 756
EventID: E569
EventTemplate: tftp: client does not accept options
Contents:
 1. tftp: client does not accept options
 2. tftp: client does not accept options
 3. tftp: client does not accept options
 4. tftp: client does not accept options
 5. tftp: client does not accept options
 6. tftp: client does not accept options
 7. tftp: client does not accept options
 8. tftp: client does not accept options
 9. tftp: client does not accept options
10. tftp: client does not accept options
--------------------------------------------------
EventID: E292
EventTemplate: warning: unable to look up <*>: No such file or directory
Contents:
 1. warning: unable to look up public/pickup: No such file or directory
 2. warning: unable to look up public/pickup: No such file or directory
 3. warning: unable to look up public/pickup: No such file or directory
 4. warning: unable to look up public/pickup: No such file or directory
 5. warning: unable to look up public/pick

KeyboardInterrupt: 